In [5]:
import nibabel as nib
import sys

import plot_functions 

sys.path.append("/src/models")
from traditional_models import *

ModuleNotFoundError: No module named 'traditional_models'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import colorsys
import random
from itertools import combinations
import multiprocessing
import threading

def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors

def apply_mask(image, mask, color, alpha=0.2):
    """Apply the given mask to the image.
    """

    # The mask is drawn on the image.
    for c in range(3):
        image[:, :, c] = np.where(mask == 1, image[:, :, c] *
                                     (1 - alpha) + alpha * color[c] * 255,
                                     image[:, :, c])
    return image

def display_image(img, masks = None, colors = [(0, 0, 1)], alpha=0.2, display = False):

    if not masks is None:

        # The image is converted to a 3-channel RGB image to allow for colored masks to be added.
        img = np.expand_dims(img, axis=2)
        img = np.dstack([img] * 3)

        # Ensure masks are  in a list
        if type(masks) != list:
            masks = [masks]

        # Make sure colors are in a list
        if type(colors) != list:
            colors = [colors]

        # If not all colors are defined, generate the remaining
        if len(colors) < len(masks):
            colors += random_colors(len(masks) - len(colors))

        # If there are multiple masks, make the intersections clear
        if len(masks) > 1:
            num_masks = len(masks)

            # Generate colors an masks for intersections by looping over every combination of masks
            for comb in combinations(range(num_masks), 2):

                # Find mask combination
                maskA = masks[comb[0]]
                maskB = masks[comb[1]]

                # Calculate the intersection
                intersect = maskA * maskB

                # Add intersection as a seperate mask
                masks.append(intersect)

                # Remove intersections from the other masks
                masks[comb[0]] = maskA - intersect
                masks[comb[1]] = maskB - intersect

                # Calculate the weighted average of the two masks
                intersect_color = tuple(np.array(colors[comb[0]]) * 0.5 + np.array(colors[comb[1]]) * 0.5)

                # Add the intersection color to the intersection mask
                colors.append(intersect_color)

        # Apply masks to image
        for mask, color in zip(masks, colors):
            img = apply_mask(img, mask, color, alpha)

        # Make sure all values are integers
        img = img.astype("int")

        if display:
            plt.imshow(img)

        else:
            return img

    else:

        if display:
            plt.imshow(img, cmap='gray')

        else:
            return img

    plt.show()


def animate_CT(img_3D, angle = 1, masks = None, colors = [(0, 0, 1)], alpha=0.2, animation_speed = 50, filename="slices"):

    # Defines the indexing of the CT_image depending on the chosen view angle.
    dimension_choice = ["[i, :, :]",
                        "[:, i, :]",
                        "[:, :, i]"][angle]

    num_slices = np.shape(img_3D)[angle]

    # Ensure masks are  in a list
    if type(masks) != list:
        masks = [masks]

    fig, ax = plt.subplots()
    ims = []
    
    for i in range(num_slices):
        img = eval("img_3D" + dimension_choice)

        # In case there is no mask we skip this part
        if not masks[0] is None:
            masks_2D = []
            for mask in masks:
                masks_2D.append(eval("mask" + dimension_choice))

        else:
            masks_2D = None

        im = ax.imshow(display_image(img, masks_2D, colors=colors, alpha=alpha, display=False), animated=True)
        ims.append([im])
        print(f"{i + 1} / 1000")


    ani = animation.ArtistAnimation(fig, ims, interval=animation_speed, blit=True,
                                    repeat_delay=1000)

    ani.save(f"{filename}.mp4")



def displaySlice(CT_image, angle, slice_depth, mask = None, alpha = 0.2, colors = (0, 0, 1)):
    """
    Cuts a 2D slice from a 3D image and displays it as a grayscale image.
    :numpy.ndarray CT_image: 3D image.
    :int angle: 0,1 or 2 represents which axis the image is viewed from.
    :int slice_depth: 0 to image depth. The index of the image along the angle axis.
    :numpy.ndarray mask: A binary array of the same dimensions as CT_image indicating
    the pixelwise segmentation
    :alpha float: A number between 0 and 1 indicating the strength of the mask color compared
    to the CT_image.
    :color tuple: A three tuple of 0 to 1 values indicating the RGB mixture of red, green and
    blue in the mask color.
    """

    # Defines the indexing of the CT_image depending on the chosen view angle.
    dimension_choice = [f"[{slice_depth}, :, :]",
                        f"[:, {slice_depth}, :]",
                        f"[:, :, {slice_depth}]"][angle]

    # Redefines the CT_image as a 2D image corresponding to the selected slice and view.
    CT_image = eval("CT_image" + dimension_choice)

    if not mask is None:
        # If a mask is provided, the corresponding slice is made into the mask
        mask = eval("mask" + dimension_choice)

    display_image(CT_image, mask, colors=colors, alpha=alpha, display=True)

In [3]:
img = nib.load("/dtu/3d-imaging-center/projects/2020_QIM_22_rat_kidney/analysis/analysis_rat3/CT_rat3_kidneyProc.nii")

mask = nib.load("/dtu/3d-imaging-center/projects/2020_QIM_22_rat_kidney/analysis/analysis_rat3/Segmentation_21-04-2021.nii.gz")

In [70]:
1-np.count_nonzero(mask)/1000000000

0.999297047

In [4]:
img = img.get_fdata()
mask = mask.get_fdata()
result = np.zeros_like(mask)
result[mask > 0] = 1

#mask2 = mask.copy()
mask2 = threshold_3d(img, 70) 

NameError: name 'threshold_3d' is not defined

In [ ]:


#display_image(img[500,:,:], [mask1, mask2], colors=[(0, 0, 1), (1, 0, 0)], alpha=0.5, display=True)
animate_CT(mask2[200:800, 300:700, :], angle=1, colors=[(0, 1, 0), (1, 0, 1)], alpha=1.0)